<a href="https://colab.research.google.com/github/aferron/Dream/blob/main/EnDex_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

As a demo, this notebook will be set up with a runnable version of the EnDex evaluation model. The model data will be uploaded into the shared drive so that it can be accessed here. 

Connecting to the drive folder:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Requirements block:

In [ ]:
!python --version

!pip install torch==1.10.1 torchvision==0.11.2 torchaudio==0.10.1
!pip install transformers==4.14.0

The sample code provided by the EnDex team to run the model:

In [ ]:
import torch
from transformers import RobertaTokenizer,RobertaForSequenceClassification
tokenizer = RobertaTokenizer.from_pretrained("roberta-large")
model_dir = '/content/drive/Shareddrives/DRAEM/endex_model'
model = RobertaForSequenceClassification.from_pretrained(model_dir, local_files_only=True)
inputs = tokenizer("it's such a great point, and i'd love to hear back on your thoughts!", return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits
predicted_class_id = logits.argmax().item()
print('the engagingness prediction of the input sentence is: ', predicted_class_id)


In [ ]:
import json
from tqdm import tqdm

class evaluate_data_set:
  def __init__(self, model) -> None:
    f = open("/content/drive/Shareddrives/DRAEM/Reflect/data/organized_Reflect_9k_responses.json")
    self.reflect_data = json.load(f)
    f.close()
    self.engaging_responses = 0
    self.total = 0

  def evaluate_response(self, response: str) -> None:
    inputs = tokenizer(response, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    self.engaging_responses += predicted_class_id
    self.total += 1

  def evaluate_all_reflect_responses(self) -> float:
    for i in tqdm(self.reflect_data):
      for num in range(1, 6):
        for response in i['responses_{num}'.format(num = num)]:
          self.evaluate_response(response)
    return self.engaging_responses/self.total

In [ ]:
print(evaluate_data_set(model).evaluate_all_reflect_responses())